# Notebook to learn Alexander Spannier cochains 

An Alexander-Spannier $k$-cochain on $\mathbb{R}^m$ is a map $f:(\mathbb{R}^m)^{k+1} \longrightarrow \mathbb{R}$. 
Given a simplex $k$-simplex $\sigma = [v_0,\cdots, v_k] \subset \R^m$, we can evaluate the cochain $f$ to get a value on $\sigma$ by taking $f(\sigma)= f(v_0,\cdots,v_k)$. This gives a way of obtaining a cochain for a simplicial complex in $\R^n$ by evaluating $f$ on all of its simplices. 

Then given a collection of simplicial complexes all embedded in $\R^m$, they all get comparable or consistent features by evaluaing the same cochain  $f$ on each simplicial complex in the collection.

\textbf{Remark for later: } The set of Alexander-Spannier cochains and alternating Alexander-Spannie cochains are homotopy equivalent so at some point we might want to consider the set of alternating cochains (on simplicial complexes these correspond to the usual cochains)

In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import coo_matrix,diags
from scipy.sparse.linalg import inv
import gudhi as gd
import copy
import random
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import torch
import torch.nn as nn
from itertools import permutations
import math


### Some useful functions 

In [2]:
## Functions for generating paths data sets


def generate_diagonal_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        x = np.sort(np.random.uniform(low=-1, high=1, size=num_pts).astype('f'))
        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        x_trans = np.random.randint(-5,5)
        y_trans = np.random.randint(-5,5)

        x_values = list(x+noise_x +x_trans)
        y_values = list(np.sin(x+noise_y)+y_trans)

        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)
        
    return Paths


def generate_antidiagonal_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        x = np.sort(np.random.uniform(low=-1, high=1, size=num_pts).astype('f'))

        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        x_trans = np.random.randint(-5,5)
        y_trans = np.random.randint(-5,5)
        x_values = list(x+noise_x +x_trans)
        y_values = list(-np.sin(x+noise_y)+y_trans)
        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)
        
    return Paths
        



def generate_circular_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        endpoint = np.random.randint(0,num_pts)
        
        sample_angles = list(np.sort(np.random.uniform(0,2*np.pi, num_pts)).astype('f'))
        angles= sample_angles[endpoint:]+ sample_angles[:endpoint]
        angles = np.array(angles)
        
        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        #x_trans = np.random.randint(-5,5)
        #y_trans = np.random.randint(-5,5)

        r = np.random.uniform(0.5, 2.5)

        x_values = r*np.cos(angles)+noise_x
        y_values = r*np.sin(angles)+noise_y

        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)

    return Paths

In [3]:
# initialise an AS cochain f as an MLP

# m = dimension of space the complex lives in 
# k = dimension of cochain 

m =2
k = 1
f = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, 1)
)

simp = torch.tensor([(1,0),(0,1)]).float() ## a 1-simplex with node values in R^2
print(simp.shape)
simp = simp.reshape(1,-1)
print(simp.shape)
a = f(simp) ## evaluate the cochain on the 1-simplex 
print(a)
print(a.shape)



torch.Size([2, 2])
torch.Size([1, 4])
tensor([[0.0661]], grad_fn=<AddmmBackward0>)
torch.Size([1, 1])


In [4]:
## we can also learn l different cochains at once by using a nn.Sequential with l outputs

l=2

F = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, l)
)

simp = torch.tensor([(1,0),(0,1)]).float() ## a 1-simplex with node values in R^2
print(simp.shape)
simp = simp.reshape(1,-1)
print(simp)
print(simp.shape)
a = F(simp)
print(a)
print(a.shape)


torch.Size([2, 2])
tensor([[1., 0., 0., 1.]])
torch.Size([1, 4])
tensor([[-0.0627,  0.0017]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])


In [5]:
## This is the main function I have issues with, it is supposed to evaluate a set of l k-cochains on a k-simplex but there is an issue when I try to actually learn
## The issue is getting the right shape of output and making things differentiable..

def cochain_eval(cochains,simplex):
    """ Evaluate a set of l k-cochains on a k-simplex """
    
    # check size of simplex is compatible with cochain
    assert simplex.shape == ((k+1),m), "dimension of simplex and cochain does not match"
    
    l = cochains[-1].out_features
    
    a = cochains(simplex.reshape(1,-1))
    return(a)

m = 2
l=2
k = 1
F = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, l)
)

simp = torch.tensor([(1,0),(0,1)]).float()
a = cochain_eval(F,simp)
print(a.shape)
print(a)

torch.Size([1, 2])
tensor([[-0.1285, -0.1476]], grad_fn=<AddmmBackward0>)


In [6]:
## A function to evaluate a set of l k-cochains on a path, this is the main function we will use to learn cochains on paths
## making it work depends on the cochain_eval function above 

def cochain_eval_path(cochains,path):
    """ Evaluate a set of l k-cochains on a simplicial complex
     simplicial complex sc as array of simplices"""
    
    simplex = torch.tensor((path[0], path[1]))  # convert path[i] to a torch tensor efficiently later
    temp = cochain_eval(cochains,simplex)
    temp.retain_grad()


    for i in range(1,path.shape[0]-1):

        simplex = torch.tensor((path[i], path[i+1]))  # convert path[i] to a torch tensor efficiently later
        
        temp = torch.cat((temp, cochain_eval(cochains,simplex)),0)
        temp.retain_grad()
       
    
    return temp


### example 

m =2
k = 1
f = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, 2)
)

p0 = generate_diagonal_paths(num_paths=10,eps = 0.2, num_pts = 5)
path = p0[0]
#print("path = ", path )
output = cochain_eval_path(f,path)
print("output = ", output )
print(output.shape)

output =  tensor([[-0.0223, -0.2099],
        [-0.0048, -0.1947],
        [ 0.0110, -0.1657],
        [ 0.0375, -0.1091]], grad_fn=<CatBackward0>)
torch.Size([4, 2])


/var/folders/w2/2q_ql79j5_s6kq_b56rg9xdc0000gn/T/ipykernel_8788/3051693023.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  simplex = torch.tensor((path[0], path[1]))  # convert path[i] to a torch tensor efficiently later


## Learning part

Here we learn to classify three classes of paths in $\R^2$, one class goes diagonally up to the right, one goes along the antiddiagonal down to the right and one is circular

In [7]:
# generate data

num_paths = 100

p0 = generate_diagonal_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)
p1 = generate_antidiagonal_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)
p2 = generate_circular_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)

# join together p0, p1, p2
paths = p0+p1+p2

# generate labels
labels = np.concatenate((np.zeros(num_paths),np.ones(num_paths),2*np.ones(num_paths))).astype('f')

# perform a one hot encoding of the labels and transform to torch
labels = torch.nn.functional.one_hot(torch.tensor(labels).to(torch.int64))


In [20]:
l=3 # three classes so three outputs 
m = 2 # the paths live in R^2
k = 1 # we deal with one simplices 

## We want to learn a set of k-cochains on the paths, we have three classes so we want to learn three k-cochains at once

F = nn.Sequential(
    nn.Linear(m*(k+1),50),
    nn.ReLU(),
    nn.Linear(50,20),
    nn.ReLU(),
    nn.Linear(20, l)
)


In [22]:
epochs = 50

batch_size = len(paths)

orig_labels = np.concatenate((np.zeros(num_paths),np.ones(num_paths),2*np.ones(num_paths)))

optim = torch.optim.SGD(F.parameters(), lr=1e-4)

criterion = nn.CrossEntropyLoss()


for e in range(epochs):

    print(e)

    ## shuffle the data
    idx = np.random.permutation(len(paths))
    paths = [paths[i] for i in idx]
    labels = labels[idx]
    orig_labels = orig_labels[idx]

    correct_pred = 0 

    # initialise loss
    l = 0

    for i in range(len(paths)):

        p = paths[i]
        y = labels[i]

        X = cochain_eval_path(F,p)
        X = torch.sum(X, dim = 0)

        
        sm = torch.nn.functional.softmax(X, dim =0) 

        loss = criterion(sm,y.float())
        loss.backward()

        l += loss.detach()
        
        # get the index of the max log-probability
        pred = sm.argmax(keepdim=True).float()
        #print("predictiton = " ,sm)
        #print("original label = ",orig_labels[i])

        if pred == orig_labels[i]: ## 
          correct_pred += 1
        
        #print("y =", y)
        #print("sm =", sm)

        #for name, param in F.named_parameters():
        #  if param.grad is not None:
        #      print(name, param.grad.sum())
        #  else:
        #      print(name, param.grad)

        optim.step()
        optim.zero_grad()

    print("Epoch :", e , "Train Accuracy ", correct_pred/len(paths), "Av. Loss: ", l/len(paths))




0
Epoch : 0 Train Accuracy  0.27 Av. Loss:  tensor(0.9847)
1
Epoch : 1 Train Accuracy  0.2733333333333333 Av. Loss:  tensor(0.9795)
2
Epoch : 2 Train Accuracy  0.26666666666666666 Av. Loss:  tensor(0.9745)
3
Epoch : 3 Train Accuracy  0.26666666666666666 Av. Loss:  tensor(0.9700)
4
Epoch : 4 Train Accuracy  0.27666666666666667 Av. Loss:  tensor(0.9659)
5
Epoch : 5 Train Accuracy  0.27 Av. Loss:  tensor(0.9621)
6
Epoch : 6 Train Accuracy  0.27 Av. Loss:  tensor(0.9585)
7
Epoch : 7 Train Accuracy  0.27 Av. Loss:  tensor(0.9554)
8
Epoch : 8 Train Accuracy  0.27 Av. Loss:  tensor(0.9524)
9
Epoch : 9 Train Accuracy  0.26666666666666666 Av. Loss:  tensor(0.9495)
10
Epoch : 10 Train Accuracy  0.26666666666666666 Av. Loss:  tensor(0.9468)
11
Epoch : 11 Train Accuracy  0.2733333333333333 Av. Loss:  tensor(0.9445)
12
Epoch : 12 Train Accuracy  0.2633333333333333 Av. Loss:  tensor(0.9421)
13
Epoch : 13 Train Accuracy  0.2733333333333333 Av. Loss:  tensor(0.9400)
14
Epoch : 14 Train Accuracy  0.266

In [15]:
# print the gradient of the parameters in F
for name, param in F.named_parameters():
    if param.grad is not None:
        print(name, param.grad.sum())
    else:
        print(name, param.grad)



0.weight tensor(0.3255)
0.bias tensor(-0.0222)
2.weight tensor(-8.4955)
2.bias tensor(-0.0255)
4.weight tensor(-7.0196)
4.bias tensor(-0.0730)
6.weight tensor(-2.4088)
6.bias tensor(-0.1100)
8.weight tensor(-1.3672e-06)
8.bias tensor(-3.1991e-07)


### Graveyard 

Some old functions I havent decided if I want to keep or no  

In [ ]:
## Evaluate one cochain on a simplex 

def cochain_eval(cochain,simplex):

    """ Evaluate a k-cochains on a k-simplex """
    
    # check size of simplex is compatible with cochain
    assert simplex.shape == ((k+1),m), "dimension of simplex and cochain does not match"
    simplex.reshape(1,-1)
    simplex = simplex.reshape(1,-1)
    out = torch.zeros(1)
    out = cochain(simplex)[0]
    print(out.shape)
    return out 


simp = torch.tensor([(1,0),(0,1)]).float()
res = cochain_eval(f,simp)
print(res.shape)
print(res)


torch.Size([1])
torch.Size([1])
tensor([0.0332], grad_fn=<SelectBackward0>)
